The Transcations PGM shown here is derived from the cancer (http://www.bnlearn.com/bnrepository/#cancer) bayesian network architecture using pgmpy and do some simple queries on the network.

In pgmpy, the general flow of defining a network is to first define the network and then add the parameters to it.
Then we will begin querying the network for results.

Note: In this example we are using discrete values for random variables Time,Amount,Suspicious,Fraud and User Error {0,1} for demonstration purposes.

In [1]:
# Import Required Packages
from pgmpy.models import BayesianModel
from pgmpy.inference import VariableElimination
from pgmpy.factors.discrete import TabularCPD
import networkx as nx
import pylab as plt


#Initialise Bayesian Transactions Model
transact_model = BayesianModel([('Time', 'Suspicious'), 
                              ('Amount', 'Suspicious'),
                              ('Suspicious', 'Fraud'),
                              ('Suspicious', 'User Error')]) 


# Display PGM using NetworkX package
nx.draw(transact_model,width = 3,node_size = 2700, font_size = 10, with_labels=True)
plt.show()

<Figure size 640x480 with 1 Axes>

In [6]:
# Creating the CPD associated with the network
cpd_time = TabularCPD(variable='Time', variable_card=2,
                      values=[[0.9], [0.1]])

cpd_amount = TabularCPD(variable='Amount', variable_card=2,
                       values=[[0.3], [0.7]])

cpd_suspicious = TabularCPD(variable='Suspicious', variable_card=2,
                        values=[[0.03, 0.05, 0.001, 0.02],
                                [0.97, 0.95, 0.999, 0.98]],
                        evidence=['Amount', 'Time'],
                        evidence_card=[2, 2])

cpd_fraud = TabularCPD(variable='Fraud', variable_card=2,
                      values=[[0.9, 0.2], [0.1, 0.8]],
                      evidence=['Suspicious'], evidence_card=[2])

cpd_user_error = TabularCPD(variable='User Error', variable_card=2,
                      values=[[0.65, 0.3], [0.35, 0.7]],
                      evidence=['Suspicious'], evidence_card=[2])

In [7]:
print(cpd_amount)

+-----------+-----+
| Amount(0) | 0.3 |
+-----------+-----+
| Amount(1) | 0.7 |
+-----------+-----+


In [8]:
print(cpd_fraud)

+------------+---------------+---------------+
| Suspicious | Suspicious(0) | Suspicious(1) |
+------------+---------------+---------------+
| Fraud(0)   | 0.9           | 0.2           |
+------------+---------------+---------------+
| Fraud(1)   | 0.1           | 0.8           |
+------------+---------------+---------------+


In [9]:
# Associating the cpds with the Bayesian Network
transact_model.add_cpds(cpd_time, cpd_amount, cpd_suspicious, cpd_fraud, cpd_user_error)

In [10]:
# Checking if the transactions cpds are valid for the model.
transact_model.check_model()

True

In [11]:
transact_model.active_trail_nodes('Amount')

{'Amount': {'Amount', 'Fraud', 'Suspicious', 'User Error'}}

In [12]:
# Doing some simple queries on the network
transact_model.is_active_trail('Amount', 'Time')

False

In [13]:
transact_model.is_active_trail('Amount', 'Time', observed=['Suspicious'])

True

In [14]:
transact_model._get_ancestors_of('Fraud')

{'Amount', 'Fraud', 'Suspicious', 'Time'}

In [15]:
transact_model.local_independencies('Fraud')

(Fraud _|_ Time, User Error, Amount | Suspicious)

In [16]:
transact_model.get_independencies()

(Time _|_ Amount)
(Time _|_ Fraud, User Error | Suspicious)
(Time _|_ User Error | Fraud, Suspicious)
(Time _|_ Fraud | Suspicious, User Error)
(Time _|_ Fraud, User Error | Suspicious, Amount)
(Time _|_ User Error | Fraud, Suspicious, Amount)
(Time _|_ Fraud | Amount, Suspicious, User Error)
(Amount _|_ Time)
(Amount _|_ Fraud, User Error | Suspicious)
(Amount _|_ User Error | Fraud, Suspicious)
(Amount _|_ Fraud, User Error | Time, Suspicious)
(Amount _|_ Fraud | Suspicious, User Error)
(Amount _|_ User Error | Fraud, Suspicious, Time)
(Amount _|_ Fraud | Time, Suspicious, User Error)
(Fraud _|_ Time, User Error, Amount | Suspicious)
(Fraud _|_ User Error, Amount | Time, Suspicious)
(Fraud _|_ Time, Amount | Suspicious, User Error)
(Fraud _|_ Time, User Error | Suspicious, Amount)
(Fraud _|_ Amount | Time, Suspicious, User Error)
(Fraud _|_ User Error | Time, Suspicious, Amount)
(Fraud _|_ Time | Amount, Suspicious, User Error)
(User Error _|_ Fraud, Time, Amount | Suspicious)
(User 

In [17]:
# Convert Bayesian Model to Junction Tree/ Clique Tree
jt = transact_model.to_junction_tree()

After constructing the PGM, we are able to do exact Inference using Variable Elimination

In [18]:
transact_infer = VariableElimination(transact_model)

# Computing the probability of Fraud given Suspicious = 1.
q_fraud = transact_infer.query(variables=['Fraud'], evidence={'Suspicious': 1})
print(q_fraud)

# Computing the probability of User Error given Suspicious = 0.
q_user_error = transact_infer.query(variables=['User Error'], evidence={'Suspicious': 0})
print(q_user_error)


Eliminating: Amount: 100%|██████████| 3/3 [00:00<00:00, 969.11it/s]

+----------+--------------+
| Fraud    |   phi(Fraud) |
+==========+==============+
| Fraud(0) |       0.2000 |
+----------+--------------+
| Fraud(1) |       0.8000 |
+----------+--------------+
+---------------+-------------------+
| User Error    |   phi(User Error) |
+===============+===================+
| User Error(0) |            0.6500 |
+---------------+-------------------+
| User Error(1) |            0.3500 |
+---------------+-------------------+
